    Surprise provides a bunch of built-in algorithms. All algorithms derive from the AlgoBase base class, where are implemented some key methods (e.g. predict, fit and test). The list and details of the available prediction algorithms can be found in the prediction_algorithms package documentation.

    Every algorithm is part of the global Surprise namespace, so you only need to import their names from the Surprise package, for example:

In [3]:
from surprise import KNNBasic
algo = KNNBasic()

    Some of these algorithms may use baseline estimates, some may use a similarity measure. We will here review how to configure the way baselines and similarities are computed.

1. Baselines estimates configuration

    This section only applies to algorithms (or similarity measures) that try to minimize the following regularized squared error (or equivalent):

    ∑rui∈Rtrain(rui−(μ+bu+bi))2+λ(b2u+b2i).
    For algorithms using baselines in another objective function (e.g. the SVD algorithm), the baseline configuration is done differently and is specific to each algorithm. Please refer to their own documentation.

    First of all, if you do not want to configure the way baselines are computed, you don’t have to: the default parameters will do just fine. If you do want to well… This is for you.

    You may want to read section 2.1 of [Kor10] to get a good idea of what are baseline estimates.

    Baselines can be estimated in two different ways:

        Using Stochastic Gradient Descent (SGD).
        Using Alternating Least Squares (ALS).
        
    You can configure the way baselines are computed using the bsl_options parameter passed at the creation of an algorithm. This parameter is a dictionary for which the key 'method' indicates the method to use. Accepted values are 'als' (default) and 'sgd'. Depending on its value, other options may be set. 

    For ALS:

        'reg_i': The regularization parameter for items. Corresponding to λ2 in [Kor10]. Default is 10.
        'reg_u': The regularization parameter for users. Corresponding to λ3 in [Kor10]. Default is 15.
        'n_epochs': The number of iteration of the ALS procedure. Default is 10. Note that in [Kor10], what is described is a single iteration ALS process.
        
    And for SGD:

        'reg': The regularization parameter of the cost function that is optimized, corresponding to λ1 and then λ5 in [Kor10] Default is 0.02.
        'learning_rate': The learning rate of SGD, corresponding to γ in [Kor10]. Default is 0.005.
        'n_epochs': The number of iteration of the SGD procedure. Default is 20.

    For both procedures (ALS and SGD), user and item biases (bu and bi) are initialized to zero.

Usage examples:

In [8]:
# From file examples/baselines_conf.py
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)

Using ALS


NameError: name 'BaselineOnly' is not defined

In [9]:
# From file examples/baselines_conf.py
print('Using SGD')
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)

Using SGD


NameError: name 'BaselineOnly' is not defined

    Note that some similarity measures may use baselines, such as the pearson_baseline similarity. Configuration works just the same, whether the baselines are used in the actual prediction r^ui or not:

In [10]:
# From file examples/baselines_conf.py
bsl_options = {'method': 'als',
               'n_epochs': 20,
               }
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(bsl_options=bsl_options, sim_options=sim_options)

This leads us to similarity measure configuration, which we will review right now.

2. Similarity measure configuration

    Many algorithms use a similarity measure to estimate a rating. The way they can be configured is done in a similar fashion as for baseline ratings: you just need to pass a sim_options argument at the creation of an algorithm. This argument is a dictionary with the following (all optional) keys:

    'name': The name of the similarity to use, as defined in the similarities module. Default is 'MSD'.
    'user_based': Whether similarities will be computed between users or between items. This has a huge impact on the performance of a prediction algorithm. Default is True.
    'min_support': The minimum number of common items (when 'user_based' is 'True') or minimum number of common users (when 'user_based' is 'False') for the similarity not to be zero. Simply put, if |Iuv|<min_support then sim(u,v)=0. The same goes for items.
    'shrinkage': Shrinkage parameter to apply (only relevant for pearson_baseline similarity). Default is 100.
    
    Usage examples:

In [4]:
sim_options = {'name': 'cosine',
              'user_based':False # compute similarities between items
              }
algo = KNNBasic(sim_options=sim_options)

In [5]:
sim_options = {'name': 'pearson_baseline',
              'shrinkage': 0 # no shrinkage
              }
algo = KNNBasic(sim_options=sim_options)